In [1]:
import streamlit as st
import time
from datetime import datetime as dt
from typing import Dict, List, Optional
import redis
import json
import uuid
import sys
from tinydb import TinyDB, Query
from datetime import datetime as dt, timedelta
import os
from typing import List, Tuple, Optional, Dict, Any
import importlib
import uuid

In [2]:
root_path = "/Users/zejunzheng/Documents/GenAI/stock_searcher"  # Absolute path
sys.path.append(root_path)

In [3]:
import redis_q.redisUtils as rq # import RQJobQ
import dbutils.db_classes as dbs
import webUtils.newsUtils as news_cls
import config.conf as configure
import LLMUtils.llmTools as llm_tools

In [4]:
insighter = llm_tools.NewsInsights(configure.paths['news_db'], 
                                   configure.paths["frag_db"],
                                   configure.paths['crop_prompt'], 
                                   configure.paths['env_path'])

In [5]:
job_queue = rq.RQJobQ()

In [1]:
while True:
    try:
        job_id, urls, ticker, org = job_queue.get_crop_job()
        
        if job_id:
            print(f"Processing job {job_id} - {ticker} -- {org} -- {len(urls)}")
            status = insighter.process_job(job_id, org, ticker, urls)
            print(f"Completed job {job_id} - {status}")
            
    except redis.exceptions.TimeoutError:
        continue
    except Exception as e:
        print(f"Error processing job: {e}")
        time.sleep(1) 

In [7]:
print("sumary jobs: ", job_queue.RQ.llen(job_queue.crop_queue_name))

sumary jobs:  0
